# Udacity Lesson 1: Finding Lanes in Video
by Tim Chen

This project uses the concepts learned in the first section to identify lanes on the road.  

We first start with the input frame which is an RGB image of (540, 960, 3):
![](lesson1/figure1.png)

It is then 'flattened' into a monochrome grayscale in order to simplify processing:
![](lesson1/figure2.png)

To remove some noise from the input, we use a Gaussian function to smooth the image to find large changes in pixel intensity that would indicate an edge. The following example has been exaggerated in order to visualize the smoothing effect (kernel=31):
![](lesson1/figure3.png) 

Using moderate threshold values (Low=1, High=100), the Canny function give a typical 'edge-detected' image: 
![](lesson1/figure4-1.png)

Tuning the threshold parameters (Low=150, High=200) to maximize high-contrasting regions, we can filter out more of the image in order to find areas of largest transitions or edges:
![](lesson1/figure4-2.png)

Next, a mask is created to remove areas (not in white) that will not be considered in detecting the space of the lane... 
![](lesson1/figure5.png)

...it is then applied to the Canny image in order to isolate the lane markings within the masked region:
![](lesson1/figure6.png)

By applying the Hough Transform, will allows us to determine which pixels that fall roughly on a line. The function will determine if enough pixels form a line based on the threshold parameters declared at the beginning of the function. Pixels that don't meet the criteria for a line are filtered out and the remaining pixels are replaced with a line. Re-drawing the image with end-points of the lines will give us a clean image we can overlay:
![](lesson1/figure7.png)

The final result of overlaying the Hough Transform image over the original input image will verify if we've correctly 'detected the lane':
![](lesson1/figure8.png)


# Reflection

Overall, this gives a basic algorithm for detecting lanes. Because of its static masks and input parameters, it might be a useful algorithm for only keeping vehicles evenly within the lane. 

### Road limitations
It will fail on curved sections of road because the road will curve outside of the the masking region. Applying a dynmic mask will help this algorithm be more effective in finding the radius of curvature for the road. Intersections and merge lanes will not also work with this algorithm since you have horizontal lines at intersecting roads and two merging lines for merging roads. Having a multi-pass algorithm to apply different filters for each scenario would help identify the lanes correctly.


### Lighting & Environmental Conditions 
This algorithm does not take into changing lighting conditions, but some dynamic contrasting might help with daytime conditions. Bright light such as sunset/sunrise might be an issue as well as low-light conditions due to the aperature of the camera. Low-visibility conditions such as fog and rain will also affect the sensitivity of the algorithm. Having a combination of sensors onboard such as infrared or polarized filters can help differentiate objects in a scene, but will require more compute power.

### Lane Markings
Markers are less of an issue since the images are converted to grayscale. Double-lines will be averaged together to form a single line. Small lane markings such as those reflective squares will be difficult since they fall under the minimum length of a line and are spaced far apart. Roads with no lane markings will also fail to be detected.


## 
Below is the code to process the "solidWhiteRight.mp4" and "solidYellowLeft.mp4" videos to apply the algorithm above to detect the lanes on a straight highway:

In [1]:
#Import packages
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from moviepy.editor import VideoFileClip
from IPython.display import HTML


def process_image(image):
    
    #GRAYSCALE & NOISE FILTERING
        #Function begins with taking the input image and converting it to grayscale 
    kernel_size = 5 #Set kernel size for Gaussian smoothing; Must be an odd number (3, 5, 7...)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #Conversion to grayscale
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0) #Gaussian smoothing function 
    
    #EDGE DETECTION
        # After 'flattening' the RGB image into grayscale and reducing noise, use the Canny function to detect edges. 
    # Threshold parameters for Canny edge detection:
    low_threshold = 150
    high_threshold = 200
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold) # Returns an image of contours/edges

    #MASKING
        # We now need to create a mask to define an area of where the lane is in the image. 
        # Ideally, this would be a dynamic mask, but for the purposes of this lesson, we create a static mask. 
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   
    
        # We use cv2.fillPoly() to remove the regions where the lane cannot be by defining a four-sided mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 320), (500, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color) # Fills verticies in the empty mask with a polygon
    masked_edges = cv2.bitwise_and(edges, mask) # Processes the Edges image and removes pixels outside the mask

    #HOUGH TRANSFORM
        # Performing the hough transform will allow us to find pixels that are roughly congregated along in lines  
    
    # Hough transform parameters
    rho = 1 # Distance resolution in pixels of the Hough grid
    theta = np.pi/180 # Angular resolution in radians of the Hough grid
    threshold = 15 # Minimum number of votes (intersections in Hough grid cell)
    min_line_length = 120 # minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    left_lane = []
    right_lane = []

    for line in lines:
        for x1,y1,x2,y2 in line:
            m_i = (y2-y1)/(x2-x1)
            if ( -.2 <= m_i <= .2 ):
                break
            elif(m_i < -.2):
                left_lane.append([x1, y1, x2, y2])
            else:
                right_lane.append([x1, y1, x2, y2])

    if not left_lane:
        return image
    elif not right_lane:
        return image
    else:
        left = np.mean(left_lane,axis=0).astype(int)
        right = np.mean(right_lane,axis=0).astype(int)

        m1 = (left[3]-left[1])/(left[2]-left[0])
        b1 = left[1]-(m1*left[0])

        m2 = (right[3]-right[1])/(right[2]-right[0])
        b2 = right[1]-(m2*right[0])

        y_max = image.shape[0]
        xL1_max = ((y_max - b1) / m1).astype(int)
        xL2_max = ((vertices[0,1,1]-b1)/m1).astype(int)
        xR1_max = ((y_max - b2) / m2).astype(int)
        xR2_max = ((vertices[0,2,1]-b2)/m2).astype(int)


        cv2.line(line_image,(xL1_max,y_max),(xL2_max,vertices[0,1,1]),(0,255,0),10)
        cv2.line(line_image,(xR2_max,vertices[0,2,1]),(xR1_max,y_max),(255,0,0),10)

        # Create a "color" binary image to combine with line image
        color_edges = np.dstack((edges, edges, edges)) 

        # Draw the lines on the edge image
        result = cv2.addWeighted(image, 1, line_image, 1, 0) 

        return result

        
%matplotlib inline

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

#HIT SHIFT-ENTER TO RUN

Imageio: 'ffmpeg.win32.exe' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.win32.exe (27.4 MB)
Downloading: 8192/28781056 bytes (0.0%)32768/28781056 bytes (0.1%)40960/28781056 bytes (0.1%)65536/28781056 bytes (0.2%)81920/28781056 bytes (0.3%)98304/28781056 bytes (0.3%)131072/28781056 bytes (0.5%)163840/28781056 bytes (0.6%)196608/28781056 bytes (0.7%)245760/28781056 bytes (0.9%)294912/28781056 bytes (1.0%)344064/28781056 bytes (1.2%)393216/28781056 bytes (1.4%)458752/28781056 bytes (1.6%)524288/28781056 bytes (1.8%

100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:37<00:00,  5.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 39.5 s


In [2]:
yellow_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

#HIT SHIFT-ENTER TO RUN

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:07<00:00, 82.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 8.27 s


## This concludes the homework assignment for Lesson 1